In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import json

from datetime import datetime, timedelta

In [15]:
# Read the Parquet file
df = pd.read_parquet('Binance_BTCUSDT_20241022_021225_10.parquet')

# Print the DataFrame
print(df.head())

                        ts                                               bids  \
0  2024-10-21 18:12:16.280  [[1000.000, 141.322], [5000.000, 6.671], [3300...   
1  2024-10-21 18:12:16.511  [[1000.000, 141.322], [5000.000, 6.691], [1000...   
2  2024-10-21 18:12:16.792  [[1000.000, 141.322], [5000.000, 6.671], [5476...   
3  2024-10-21 18:12:17.097  [[1000.000, 141.322], [5000.000, 6.691], [2000...   
4  2024-10-21 18:12:17.295  [[1000.000, 141.322], [5000.000, 6.671], [3300...   

                                                asks  
0  [[68461.000, 1.701], [68474.000, 0.714], [6847...  
1  [[68461.000, 1.714], [68462.400, 0.004], [6847...  
2  [[68461.000, 1.716], [68471.300, 0.770], [6847...  
3  [[68461.000, 1.636], [68462.400, 0.002], [6846...  
4  [[68461.000, 1.651], [68462.400, 0.004], [6847...  


In [16]:
df

,ts,bids,asks
0,2024-10-21 18:12:16.280,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[68461.000, 1.701], [68474.000, 0.714], [6847..."
1,2024-10-21 18:12:16.511,"[[1000.000, 141.322], [5000.000, 6.691], [1000...","[[68461.000, 1.714], [68462.400, 0.004], [6847..."
2,2024-10-21 18:12:16.792,"[[1000.000, 141.322], [5000.000, 6.671], [5476...","[[68461.000, 1.716], [68471.300, 0.770], [6847..."
3,2024-10-21 18:12:17.097,"[[1000.000, 141.322], [5000.000, 6.691], [2000...","[[68461.000, 1.636], [68462.400, 0.002], [6846..."
4,2024-10-21 18:12:17.295,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[68461.000, 1.651], [68462.400, 0.004], [6847..."
5,2024-10-21 18:12:17.609,"[[1000.000, 141.322], [5000.000, 6.711], [6846...","[[68461.000, 1.665], [68462.400, 0.004], [6846..."
6,2024-10-21 18:12:17.814,"[[1000.000, 141.322], [5000.000, 6.691], [3300...","[[68461.000, 1.356], [68462.400, 0.004], [6846..."
7,2024-10-21 18:12:18.124,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[68461.000, 1.360], [68462.400, 0.002], [6846..."
8,2024-10-21 18:12:18.325,"[[1000.000, 141.322], [5000.000, 6.691], [3300...","[[68461.000, 1.348], [68462.400, 0.004], [6847..."
9,2024-10-21 18:12:18.632,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[68461.000, 1.586], [68461.800, 0.102], [6846..."
